In [1]:
import pandas as pd
import mysql.connector
from datetime import datetime

# Step 1: Load Data
file_path = r"C:\Users\Digital Suppliers\Desktop\Projects of Mine\police\Police_Dataset - file.csv"
df = pd.read_csv(file_path)

In [2]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,1/2/2005,1:55,NaN,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,1/18/2005,8:15,NaN,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,1/23/2005,23:15,NaN,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2/20/2005,17:15,NaN,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,3/14/2005,10:00,NaN,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65530,12/6/2012,17:54,NaN,F,1987.0,25.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
65531,12/6/2012,22:22,NaN,M,1954.0,58.0,White,Speeding,Speeding,False,NaN,Warning,False,0-15 Min,False
65532,12/6/2012,23:20,NaN,M,1985.0,27.0,Black,Equipment/Inspection Violation,Equipment,False,NaN,Citation,False,0-15 Min,False
65533,12/7/2012,0:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False


In [3]:
# Step 2: Data Cleaning
# Drop columns with all missing values
df.dropna(axis=1, how='all', inplace=True)

# Fill NaNs with suitable values
df.fillna({
    'driver_age': df['driver_age'].median(),
    'search_type': 'None',
    'stop_duration': 'Unknown',
    'violation': 'Unknown',
    'stop_outcome': 'Unknown',
}, inplace=True)

# Convert date & time to timestamp
df['timestamp'] = pd.to_datetime(df['stop_date'] + ' ' + df['stop_time'])

In [2]:
import pymysql

In [4]:
# Connect to MySQL
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="1234",
 
)

cursor = conn.cursor()
cursor.execute("create database securecheck")


1

In [8]:

cursor.execute("USE securecheck;")

0

In [12]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS check_post_logs(
        id INT AUTO_INCREMENT PRIMARY KEY,
        vehicle_number VARCHAR(20),
        driver_id INT,
        officer_id INT,
        timestamp TIMESTAMP,
        status VARCHAR(50)
    )
""")

print("Table created successfully!")

Table created successfully!


In [ ]:
##Officers Table
cursor.execute("""CREATE TABLE IF NOT EXISTS officers (
    officer_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    post_location VARCHAR(100),
    shift_time VARCHAR(50)"
    """)

In [ ]:





# Step 3: Insert Data

# Insert Vehicles Data
for _, row in df.iterrows():
    try:
        cursor.execute("""
            INSERT IGNORE INTO vehicles (vehicle_number, owner_id, type, flagged_status)
            VALUES (%s, %s, %s, %s)
        """, (
            row['vehicle_number'] if 'vehicle_number' in row else f'VEH{row.name}',
            row.name,  # owner_id (for simplicity using index)
            'Private',  # assuming
            bool(row['drugs_related_stop'])
        ))
    except Exception as e:
        print(f"Error inserting vehicle: {e}")

# Insert Officers Data (dummy data for demo)
officers = [
    ('John Doe', 'Checkpoint A', 'Morning'),
    ('Jane Smith', 'Checkpoint B', 'Night'),
    ('Mike Johnson', 'Checkpoint C', 'Evening')
]

for officer in officers:
    cursor.execute("""
        INSERT IGNORE INTO officers (name, post_location, shift_time)
        VALUES (%s, %s, %s)
    """, officer)

# Insert Violations Data
for _, row in df.iterrows():
    try:
        cursor.execute("""
            INSERT INTO violations (vehicle_number, violation_type, fine_amount)
            VALUES (%s, %s, %s)
        """, (
            row['vehicle_number'] if 'vehicle_number' in row else f'VEH{row.name}',
            row['violation'],
            500.00  # default fine
        ))
    except Exception as e:
        print(f"Error inserting violation: {e}")

# Insert Check Post Logs Data
for _, row in df.iterrows():
    try:
        cursor.execute("""
            INSERT INTO check_post_logs (vehicle_number, driver_id, officer_id, timestamp, status)
            VALUES (%s, %s, %s, %s, %s)
        """, (
            row['vehicle_number'] if 'vehicle_number' in row else f'VEH{row.name}',
            row.name,  # driver_id (using index as dummy)
            1,  # assuming officer ID 1
            row['timestamp'],
            row['stop_outcome']
        ))
    except Exception as e:
        print(f"Error inserting log: {e}")

conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")


: 